# Database - functions for data back-end / manipulations

This is using an alternate approach:
  - Export all of my Apple HealthFit data from the Health app to export.zip 
  - Converted this to a SQLite database using `healthfit-to-sqlite`
 
  Queries can then against this database to build the cache file (or possibly a smaller custom SQLite file.

## TODO
* This is still a work in progress
* Need to write the queries to marshall the data for each of the workouts within each group of walks
* Then cache this data - maybe try another (small) sqlite db for the caching (instead of feather)
* **NOTE: It looks like the queries are being truncated at 1000 values - need to fix**


In [2]:

import pandas as pd

from dateutil.parser import parse
import datetime as dt
import sqlite3 as sql
from pathlib import Path
import tomli

import subprocess
import pendulum
from sqlite_utils import Database
import reverse_geocoder as rg
import healthkit-to-sqlite

### Datasette.io approach

#### Exporting HealthKit data / creating SQLite DB

First export HealthKit data using the Health app - select your profile icon from the top-right of the main screen and then select **Export All Health Data** (this can take some time to create the `export.zip` file).

The archive can be converted to a SQLite database using the following command:

`healthkit-to-sqlite export.zip healthkit_db.sqlite`

In [4]:
# HEALTHKIT_DATA_PATH = Path("/Users/mjboothaus/data/healthkit")

HEALTHKIT_DATA_PATH = Path("/Users/mjboothaus/icloud/Data/apple_health_export")
export_zip = HEALTHKIT_DATA_PATH / "export.zip"

In [5]:
def convert_healthkit_export_to_sqlite(export_zip):
    zip_file = export_zip.as_posix()
    if export_zip.exists() == False:
        print(zip_file, ": not found")
        return None, f"{zip_file}: not found"
    zip_file_date = pendulum.instance(dt.datetime.fromtimestamp(export_zip.stat().st_ctime))

    db_file = zip_file.replace("export.zip", "healthkit_db.sqlite")
    if Path(db_file).exists() == True:
        Path(db_file).unlink()
    sp_cmd = f"pipx run healthkit-to-sqlite {zip_file} {db_file}"
    print(sp_cmd)
    print('---------------------------------------------------------------------------------------------')
    print('Please wait: converting healthkit export.zip to sqlite database (takes just over a minute)...')

    sp = subprocess.Popen(sp_cmd, stdout=subprocess.PIPE, shell=True)
    (sp_output, sp_err) = sp.communicate()  

    #This makes the wait possible
    sp_status = sp.wait()

    db_file_with_date = db_file.replace(".sqlite", "_" + zip_file_date.to_date_string().replace("-", "_") + ".sqlite")

    export_zip.rename(zip_file.replace(".zip", "_" + zip_file_date.to_date_string().replace("-", "_") + ".zip"))
    Path(db_file).rename(db_file_with_date)

    return db_file_with_date, sp_output

In [6]:
db_file, output = convert_healthkit_export_to_sqlite(export_zip)

pipx run healthkit-to-sqlite /Users/mjboothaus/icloud/Data/apple_health_export/export.zip /Users/mjboothaus/icloud/Data/apple_health_export/healthkit_db.sqlite
---------------------------------------------------------------------------------------------
Please wait: converting healthkit export.zip to sqlite database (takes just over a minute)...


creating virtual environment...
installing healthkit-to-sqlite...


In [8]:
db_file

'/Users/mjboothaus/icloud/Data/apple_health_export/healthkit_db_2022_04_28.sqlite'

In [10]:
def create_df_from_sql_query_in_file(filename_dot_sql, conn, parse_dates):
# Read the sql file
    query_file = get_project_root_alternate() / "sql" / filename_dot_sql

    with open(query_file, 'r') as query:
        # connection == the connection to your database
        sql_text = query.read()
        print(sql_text)
        df = pd.read_sql_query(sql_text, conn, parse_dates=parse_dates)
    return df

In [11]:
db = Database(db_file)

In [12]:
workouts_df = create_df_from_sql_query_in_file("select_star_walking_workouts.sql", db.conn, ['startDate', 'endDate'])

select
    id as workout_id,
    duration as duration_minutes,
    totaldistance as totaldistance_km,
    totalenergyburned as totalenergyburned_kJ,
    sourcename,
    sourceversion,
    startdate,
    enddate,
    metadata_hkweathertemperature,
    metadata_hkweatherhumidity,
    metadata_hkelevationascended,
    metadata_hkaveragemets
from
    workouts
where workoutactivitytype = "HKWorkoutActivityTypeWalking" order by id


 /* Excluded fields:
    workoutactivitytype,   # just walking
    durationunit,          # fixed - min
    totaldistanceunit,     # fixed - km
    totalenergyburnedunit, # fixed - kJ
    device,
    creationdate,          # not really of interest (start date instead)
    workout_events,        # think this is redundant info (need to check - JSON?)
    metadata_hkgroupfitness,
    metadata_hkworkoutbrandname,
    metadata_hktimezone,
    metadata_hkcoachedworkout,
    metadata_hkwasuserentered,
    metadata_hkindoorworkout,
    metadata_hkelevationascended,
    m

In [13]:
workouts_df.head()

,workout_id,duration_minutes,totaldistance_km,totalenergyburned_kJ,sourceName,sourceVersion,startDate,endDate,metadata_HKWeatherTemperature,metadata_HKWeatherHumidity,metadata_HKElevationAscended,metadata_HKAverageMETs
0,00622abc5705b0b7ce1019a8778f26f937e130bf,38.84239016572634,2.582174845324617,1056.937420549035,Michael and Ai Leen’s Apple Watch,6.1.1,2020-03-18 14:06:05+10:00,2020-03-19 14:00:17+10:00,78 degF,4400 %,4350 cm,4.53858 kcal/hr·kg
1,006282a0c6085c721ba9179b8eee64781b435bce,10.44971075057983,0.4370095767620951,259.2757187516958,Michael and Ai Leen’s Apple Watch,8.0,2021-10-15 07:19:03+10:00,2021-10-15 07:29:30+10:00,57.2 degF,7300 %,870 cm,4.4021 kcal/hr·kg
2,008ba174785dd6056b5ca1ec10078acdd7802c80,47.70666061639785,3.533976854936286,1187.535097164499,Michael and Ai Leen’s Apple Watch,5.2.1,2019-08-27 10:57:10+10:00,2019-08-27 11:44:52+10:00,61 degF,7500 %,1660 cm,None
3,00a5b19aad5ac7e38a951442cc38e92ca83e1c75,18.51035615603129,1.479545586945489,500.3953012249446,Michael and Ai Leen’s Apple Watch,5.1.3,2019-03-25 12:59:14+10:00,2019-03-25 13:17:44+10:00,None,None,None,None
4,00c93d495a03c7ba98bc6db89f90cab645a18c90,50.61882083415985,2.702121313092168,984.2600600944362,Michael and Ai Leen’s Apple Watch,6.2.6,2020-07-02 16:28:41+10:00,2020-07-02 17:19:18+10:00,52 degF,6200 %,1970 cm,3.60381 kcal/hr·kg


In [14]:
# Trying to find the start point in each walk workout -- as date is not a date field in db not clear if sort by date and limit 1 will work
# might need to import table to pandas convert types and then export to db before doing query. Else use sqlite_utils to change column types.

start_point_df = create_df_from_sql_query_in_file("select_start_point_workout.sql", db.conn, ['date'])

-- Get starting point for each workout
select
  start_datetime,
  start_latitude,
  start_longitude,
  start_altitude,
  start_speed,
  workout_id
from
  (
    select
      date as start_datetime,
      latitude as start_latitude,
      longitude as start_longitude,
      altitude as start_altitude,
      speed as start_speed,
      workout_id,
      row_number() over (
        partition by workout_id
        order by
          date asc
      ) as date_rank
    from
      workout_points
  )
where
  date_rank = 1


In [15]:
start_point_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   start_datetime   771 non-null    object 
 1   start_latitude   771 non-null    float64
 2   start_longitude  771 non-null    float64
 3   start_altitude   771 non-null    float64
 4   start_speed      771 non-null    float64
 5   workout_id       771 non-null    object 
dtypes: float64(4), object(2)
memory usage: 36.3+ KB


In [16]:
start_point_df.head()

,start_datetime,start_latitude,start_longitude,start_altitude,start_speed,workout_id
0,2020-03-18T04:06:07Z,-33.834663,151.146122,33.426914,1.477171,00622abc5705b0b7ce1019a8778f26f937e130bf
1,2021-10-14T21:19:01Z,-33.768691,150.619703,166.526031,0.106720,006282a0c6085c721ba9179b8eee64781b435bce
2,2019-08-27T00:57:17Z,-33.857683,151.158855,-0.112423,1.282526,008ba174785dd6056b5ca1ec10078acdd7802c80
3,2019-03-25T03:09:52Z,-33.849331,151.157116,17.284760,1.363591,00a5b19aad5ac7e38a951442cc38e92ca83e1c75
4,2020-07-02T06:29:57Z,-35.299718,149.123614,567.276245,0.172744,00c93d495a03c7ba98bc6db89f90cab645a18c90


In [17]:
finish_point_df = create_df_from_sql_query_in_file("select_finish_point_workout.sql", db.conn, ['date'])

-- Get finishing point for each workout
select
  finish_datetime,
  finish_latitude,
  finish_longitude,
  finish_altitude,
  finish_speed,
  workout_id
from
  (
    select
      date as finish_datetime,
      latitude as finish_latitude,
      longitude as finish_longitude,
      altitude as finish_altitude,
      speed as finish_speed,
      workout_id,
      row_number() over (
        partition by workout_id
        order by
          date desc
      ) as date_rank
    from
      workout_points
  )
where
  date_rank = 1


In [18]:
finish_point_df.head()

,finish_datetime,finish_latitude,finish_longitude,finish_altitude,finish_speed,workout_id
0,2020-03-18T04:44:34Z,-33.846918,151.153274,20.513332,0.000000,00622abc5705b0b7ce1019a8778f26f937e130bf
1,2021-10-14T21:29:29Z,-33.768667,150.619710,172.390900,0.051807,006282a0c6085c721ba9179b8eee64781b435bce
2,2019-08-27T01:44:51Z,-33.857442,151.159249,15.319762,1.354748,008ba174785dd6056b5ca1ec10078acdd7802c80
3,2019-03-25T03:17:43Z,-33.847052,151.152803,20.752979,0.956974,00a5b19aad5ac7e38a951442cc38e92ca83e1c75
4,2020-07-02T07:19:18Z,-35.299912,149.123778,566.349731,0.304687,00c93d495a03c7ba98bc6db89f90cab645a18c90


In [19]:
walk_info_df = start_point_df.merge(finish_point_df, how='inner', on='workout_id')

In [20]:
def get_location(latitude, longitude):
    location = rg.search((latitude, longitude))
    return [location[0]['name'], location[0]['admin1'], location[0]['cc']]

In [21]:
walk_info_df['start_location'] = walk_info_df.apply(lambda row: get_location(float(row['start_latitude']), float(row['start_longitude'])), axis=1)

Loading formatted geocoded file...


In [22]:
walk_info_df['finish_location'] = walk_info_df.apply(lambda row: get_location(float(row['finish_latitude']), float(row['finish_longitude'])), axis=1)

In [23]:
walk_info_df.head(10)

,start_datetime,start_latitude,start_longitude,start_altitude,start_speed,workout_id,finish_datetime,finish_latitude,finish_longitude,finish_altitude,finish_speed,start_location,finish_location
0,2020-03-18T04:06:07Z,-33.834663,151.146122,33.426914,1.477171,00622abc5705b0b7ce1019a8778f26f937e130bf,2020-03-18T04:44:34Z,-33.846918,151.153274,20.513332,0.000000,"[Hunters Hill, New South Wales, AU]","[Canada Bay, New South Wales, AU]"
1,2021-10-14T21:19:01Z,-33.768691,150.619703,166.526031,0.106720,006282a0c6085c721ba9179b8eee64781b435bce,2021-10-14T21:29:29Z,-33.768667,150.619710,172.390900,0.051807,"[Glenbrook, New South Wales, AU]","[Glenbrook, New South Wales, AU]"
2,2019-08-27T00:57:17Z,-33.857683,151.158855,-0.112423,1.282526,008ba174785dd6056b5ca1ec10078acdd7802c80,2019-08-27T01:44:51Z,-33.857442,151.159249,15.319762,1.354748,"[Drummoyne, New South Wales, AU]","[Drummoyne, New South Wales, AU]"
3,2019-03-25T03:09:52Z,-33.849331,151.157116,17.284760,1.363591,00a5b19aad5ac7e38a951442cc38e92ca83e1c75,2019-03-25T03:17:43Z,-33.847052,151.152803,20.752979,0.956974,"[Drummoyne, New South Wales, AU]","[Canada Bay, New South Wales, AU]"
4,2020-07-02T06:29:57Z,-35.299718,149.123614,567.276245,0.172744,00c93d495a03c7ba98bc6db89f90cab645a18c90,2020-07-02T07:19:18Z,-35.299912,149.123778,566.349731,0.304687,"[Forrest, Australian Capital Territory, AU]","[Forrest, Australian Capital Territory, AU]"
5,2021-10-27T18:28:26Z,-33.847081,151.152731,18.055738,0.067489,00eb7fa7b28b194d6aff23c678b7e7aa70b3c0dd,2021-10-27T21:40:15Z,-33.852263,151.153855,39.158474,0.035820,"[Canada Bay, New South Wales, AU]","[Drummoyne, New South Wales, AU]"
6,2019-12-15T23:51:54Z,-33.859273,151.154808,7.459394,1.334558,01028264001e7a35b635fa9eb76858a472cb37c2,2019-12-16T00:47:54Z,-33.858716,151.154779,12.775998,0.139416,"[Drummoyne, New South Wales, AU]","[Drummoyne, New South Wales, AU]"
7,2019-12-31T21:22:29Z,-33.859420,151.156248,15.434346,1.182388,01107b537742e1b396d742d80f26557b94de4459,2019-12-31T22:20:56Z,-33.858739,151.156270,9.560465,0.220545,"[Drummoyne, New South Wales, AU]","[Drummoyne, New South Wales, AU]"
8,2020-06-01T20:33:49Z,-33.847179,151.152555,15.193020,0.265315,013c71df802799519de191b7ff3b7dc5ffc644ba,2020-06-01T20:35:37Z,-33.847126,151.152706,18.673912,1.057088,"[Canada Bay, New South Wales, AU]","[Canada Bay, New South Wales, AU]"
9,2021-07-21T20:37:01Z,-33.852012,151.154787,34.225677,0.638369,01d750859db83e665a8113e0f0054d11f3510fff,2021-07-22T03:07:53Z,-33.847201,151.152812,23.093376,1.081762,"[Drummoyne, New South Wales, AU]","[Canada Bay, New South Wales, AU]"


In [24]:
def calculate_elapsed_time_minutes(finish_datetime, start_datetime):
    dt = pendulum.parse(finish_datetime) - pendulum.parse(start_datetime)
    return float(dt.in_seconds() / 60 / 60)

In [25]:
walk_info_df['elapsed_time_hours'] = walk_info_df.apply(lambda row: calculate_elapsed_time_minutes(row['finish_datetime'], row['start_datetime']), axis=1)

In [26]:
walk_info_df['start_datetime'] = walk_info_df['start_datetime'].apply(lambda dt: pendulum.parse(dt, tz="Australia/Sydney").to_datetime_string())    # TODO: Need to convert from UTC to Sydney local time?

In [27]:
walk_info_df.head()

,start_datetime,start_latitude,start_longitude,start_altitude,start_speed,workout_id,finish_datetime,finish_latitude,finish_longitude,finish_altitude,finish_speed,start_location,finish_location,elapsed_time_hours
0,2020-03-18 04:06:07,-33.834663,151.146122,33.426914,1.477171,00622abc5705b0b7ce1019a8778f26f937e130bf,2020-03-18T04:44:34Z,-33.846918,151.153274,20.513332,0.000000,"[Hunters Hill, New South Wales, AU]","[Canada Bay, New South Wales, AU]",0.640833
1,2021-10-14 21:19:01,-33.768691,150.619703,166.526031,0.106720,006282a0c6085c721ba9179b8eee64781b435bce,2021-10-14T21:29:29Z,-33.768667,150.619710,172.390900,0.051807,"[Glenbrook, New South Wales, AU]","[Glenbrook, New South Wales, AU]",0.174444
2,2019-08-27 00:57:17,-33.857683,151.158855,-0.112423,1.282526,008ba174785dd6056b5ca1ec10078acdd7802c80,2019-08-27T01:44:51Z,-33.857442,151.159249,15.319762,1.354748,"[Drummoyne, New South Wales, AU]","[Drummoyne, New South Wales, AU]",0.792778
3,2019-03-25 03:09:52,-33.849331,151.157116,17.284760,1.363591,00a5b19aad5ac7e38a951442cc38e92ca83e1c75,2019-03-25T03:17:43Z,-33.847052,151.152803,20.752979,0.956974,"[Drummoyne, New South Wales, AU]","[Canada Bay, New South Wales, AU]",0.130833
4,2020-07-02 06:29:57,-35.299718,149.123614,567.276245,0.172744,00c93d495a03c7ba98bc6db89f90cab645a18c90,2020-07-02T07:19:18Z,-35.299912,149.123778,566.349731,0.304687,"[Forrest, Australian Capital Territory, AU]","[Forrest, Australian Capital Territory, AU]",0.822500


In [28]:
walk_info_df = walk_info_df.merge(workouts_df, how="inner", on="workout_id")

In [29]:
walk_info_df['startDate'] = walk_info_df['startDate'].apply(lambda dt: pendulum.instance(dt).to_datetime_string()) 
walk_info_df['endDate'] = walk_info_df['endDate'].apply(lambda dt: pendulum.instance(dt).to_datetime_string())

In [30]:
walk_info_df.to_excel('walk_info_df.xlsx', index=False)

In [31]:
start_point_df['workout_id'].nunique() / len(start_point_df)

1.0

In [32]:
workouts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype                                
---  ------                         --------------  -----                                
 0   workout_id                     681 non-null    object                               
 1   duration_minutes               681 non-null    object                               
 2   totaldistance_km               681 non-null    object                               
 3   totalenergyburned_kJ           681 non-null    object                               
 4   sourceName                     681 non-null    object                               
 5   sourceVersion                  681 non-null    object                               
 6   startDate                      681 non-null    datetime64[ns, pytz.FixedOffset(600)]
 7   endDate                        681 non-null    datetime64[ns, pytz.FixedOffset(6

In [ ]:
workouts_df_cleaned = klib.data_cleaning(workouts_df)

In [ ]:
workouts_df_cleaned.info()

In [ ]:
workouts_df_cleaned.head()

In [ ]:
db_file

In [ ]:
workouts_csv = HEALTHKIT_DATA_PATH / "workouts.csv"

In [ ]:
workouts_df_cleaned.to_csv(workouts_csv, index=False)

In [ ]:
workouts_df_cleaned.columns

In [ ]:
workouts_df_cleaned.nunique()

In [ ]:
[col for col in workouts_df.columns.values if col not in workouts_df_cleaned.columns.values]

In [ ]:
tables_df = create_df_from_sql_query_in_file("list_all_tables.sql", db.conn)

In [ ]:
tables_df

In [ ]:
WALK_DETAILS_FILE = 'walk_details.toml'
walk_details = Path('../' + WALK_DETAILS_FILE)


In [ ]:
walk_details

In [ ]:
with open(walk_details, encoding="utf-8") as f:
    walk_details_dict = tomli.load(f)

In [ ]:
walk_details_dict

In [ ]:
pd.DataFrame(walk_details_dict, )

In [ ]:

def calc_walk_stats(walk_data):
    total_time = dt.timedelta(0)
    total_distance = 0

    for hike in walk_data:
        total_time += hike.index.max()
        # print(iHike+1, walk_date[iHike], hike.index.max(), hike['dist'].max() / 1e3)
        total_distance += hike['dist'].max()
    total_distance /= 1e3

    start_coord = walk_data[0][['lat', 'lon']].iloc[0].tolist()
    end_coord = walk_data[-1][['lat', 'lon']].iloc[-1].tolist()
    return total_time, total_distance, start_coord, end_coord

In [ ]:
def create_walk_cached_data_for_app(db_file, n_rows_used=5):
    # read in all of the walks data and sample at an appropriate frequency and cache for faster use in the app
    db_conn = sql.connect(db_file)
    walk_df = pd.read_sql_query('SELECT * FROM walks', db_conn)

    UNUSED_COLUMNS = ['dist', 'speed']

    walk_df.drop(UNUSED_COLUMNS, axis=1, inplace=True)
    walk_df.dropna(inplace=True)      # TODO: Check why there are a few NaNs
    walk_df = walk_df.iloc[::n_rows_used].reset_index()    # downsample

    walk_df.to_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))
    
    return walk_df

In [ ]:
# Not working yet -- this is the alternate approach to using the individual .FIT files
# walk_df = create_walk_cached_data_for_app(db_file, 10)

In [ ]:
# walk_df[walk_df['lat'].isna()]

In [ ]:
Path(db_file.as_posix().replace('.db', '.cache.feather'))

In [ ]:
# walk_df = pd.read_feather(Path(db_file.as_posix().replace('.db', '.cache.feather')))

In [ ]:
# walk_df.info()

In [ ]:
# walk_df['WalkName'].unique()